### Input-Output Operations

*Some snippets in this notebook are taken from BLG458E Functional Programming course slides which can be accessed through the following link: https://www.slideshare.net/uyar/tag/blg458e*

**IO a**: IO actions/programs of type a  
some primitive IO programs are already defined in Haskell  

#### Reading an input  
**getLine** function is used to read a line from std input

#### One-Element Type ()
If IO operation doesn't produce a result, we use IO () which returns ().  
This is like the **void** return type in C.

#### Main Module
When compiling a Haskell program, GHC creates an executable that runs the main function: **main :: IO t**  
The type **t** is generally ()

#### Printing a string
To print a string, we use the built-in function **putStr :: String -> IO ()** which takes in a string and returns the IO object that prints the string.  
To put a new line at the end automatically, use **putStrLn**

In [1]:
helloWorld :: IO ()
helloWorld = putStr "Hello, World!"

In [2]:
helloWorld

Hello, World!

In [3]:
hello :: String -> IO ()
hello name = putStr ("Hello, " ++ name ++ "!")

In [4]:
hello "Mustafa"

Hello, Mustafa!

In [5]:
putStrLn "Hello"

Hello

#### String Conversion
From a type to string, use **show**

In [6]:
show 14
show 1.23

"14"

"1.23"

From string to another type, use **read**

In [7]:
read "42" :: Integer

42

In [8]:
read "4" :: Float
read "3.14" :: Float

4.0

3.14

#### Action Sequences
IO operations consist of actions performed in a sequence  
In order to create an action sequence, use **do**  
do operation is like an imperative block where we define the steps one by one.

**Example**
putStrLn in prelude is defined as below.  
```haskell 
putStrLn :: String -> IO ()
putStrLn str = do putStr str
                  putStr "\n"```

In [9]:
putStr3Times :: String -> IO ()
putStr3Times str = do putStrLn str
                      putStrLn str
                      putStrLn str

In [10]:
putStr3Times "lol"

lol
lol
lol

#### Reading an input (without capturing the values)

In [11]:
read2Lines :: IO()
read2Lines = do getLine
                getLine
                putStrLn "2 lines read"
                
-- I know, this function makes no sense. But you got the point!

#### Reading an input with capturing the values
Value is captured using "<-"  
*Note: the values that are read can only be used in the do block*

In [12]:
helloSomebody :: IO()
helloSomebody = do name <- getLine
                   putStrLn ("Hello, " ++ name)

In [13]:
-- function that reads a line and prints it in reverse order
reverseInput :: IO ()
reverseInput = do str <- getLine
                  putStrLn (reverse str)

You can also make *local* definition using **let** that can only be used within the scope

In [14]:
-- function that reads 2 lines and prints them in reverse order
reverse2Lines :: IO ()
reverse2Lines = do line1 <- getLine  -- beware! it's NOT line1 = getLine!
                   line2 <- getLine
                   let reverse1 = reverse line1
                   let reverse2 = reverse line2
                   putStrLn (reverse1 ++ "\n" ++ reverse2)

#### Returning the Input Read from StdIn

In [15]:
getInteger :: IO Integer
getInteger = do line <- getLine  -- get the line
                return (read line :: Integer)   -- convert it to an integer
                
-- this returns the integer read from standart input

#### Loops and Recursion


In [16]:
-- function that prints what it reads recursively
copyInfinitely :: IO ()
copyInfinitely = do line <- getLine
                    putStrLn line
                    copyInfinitely

In [17]:
--  function that prints what it reads recursively N times
copyNtimes :: Integer -> IO ()
copyNtimes n
  | n == 0    = return ()
  | otherwise = do line <- getLine
                   putStrLn line
                   copyNtimes (n-1)

In [18]:
-- function that prints what it reads until the input is empty string
copyUntilEmpty :: IO ()
copyUntilEmpty = do line <- getLine
                    if line == ""
                    then return ()
                    else do putStr line
                            copyUntilEmpty

### Rock - Paper - Scissors 
In this section, a simple version of rock-paper-scissors game is implemented.

In [ ]:
-- define move types
data Move = Rock | Paper | Scissors
            deriving Show
            
-- define a type for a match 
-- with 2 lists, 1st: moves of player1, 2nd: moves of player2
type Match = ([Move], [Move])

-- define outcome function
-- 1: A wins, -1: B wins, 0: Tie
outcome :: Move -> Move -> Integer
outcome moveA moveB = case (moveA, moveB) of
    (Paper, Rock)     ->  1
    (Scissors, Rock)  -> -1
    (Rock, Paper)     -> -1
    (Scissors, Paper) ->  1
    (Rock, Scissors)  ->  1
    (Paper, Scissors) -> -1
    _                 ->  0

-- define showRound function (toString for a round)
showRound :: Move -> Move -> String
showRound moveA moveB = "A: " ++ (show moveA) ++ "B: " ++ (show moveB)

-- define a strategy
-- here are some examples

-- always play Rock
rock :: [Move] -> Move
rock     _ = Rock

-- cycle through options
cycled :: [Move] -> Move
cycled moves = case (length moves) `mod` 3 of 
  0  -> Paper
  1  -> Scissors
  2  -> Rock

-- play what opponent played last
echo :: [Move] -> Move
echo []   = Rock
echo ms   = head ms

-- interactive game
-- A: human    B: computer (playing echo)

-- convert a char into a move
convertToMove :: Char -> Move
convertToMove c
  | c `elem` "rR"  = Rock  -- elem checks if c is in the list
  | c `elem` "sS"  = Scissors
  | c `elem` "pP"  = Paper
  | otherwise      = error "Invalid move"
  
-- create the interactive game
playRound :: Match -> IO ()
playRound match@(movesA, movesB) = do
    ch <- getChar  -- get the move from user
    putStrLn ""    -- put a new line
    if ch == '.'   -- if input is '.', show the result of the match
       then putStrLn (showResult match)
       else do let moveA = convertToMove ch
               let moveB = echo movesA
               putStrLn (showRound movesA movesB)
               playRound (moveA : movesA, moveB : movesB)
               
playInteractive :: IO ()
playInteractive = playRound ([], [])


-- how about an automated version of the game?
generateMatch :: Integer -> Match
generateMatch 0 = ([], [])
generateMatch n = step generateMatch (n-1)
  where 
    step :: Match -> Match
    step (movesA, movesB) = (cycled movesB : movesA, echo movesA : movesB)